In [2]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [3]:
import numpy as np
import edt
import pathlib
from pathlib import Path
from skimage import io
import matplotlib.pyplot as plt
import torch
%matplotlib qt

In [4]:
#DATA_DIR = Path('/home/pk/Documents/rtseg/data/test_dump')
DATA_DIR = Path('/home/pk/Documents/rtseg/data/cellseg/omnipose/')
#mask_path = DATA_DIR / Path('img_000000029.png')
mask_path = DATA_DIR / Path('train_masks/233_img_masks.png')

In [5]:
mask_path

PosixPath('/home/pk/Documents/rtseg/data/cellseg/omnipose/train_masks/233_img_masks.png')

In [6]:
image = io.imread(mask_path).astype(np.float32)

In [7]:
image

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [8]:
from rtseg.cellseg.numerics.sdf_vf import sdf_vector_field
from rtseg.cellseg.numerics.affinity_vf import affinity_vector_field

In [9]:
labels = torch.from_numpy(image)[None, :]

In [10]:
labels.ndim

3

In [11]:
vf = sdf_vector_field(labels, kernel_size=15)

In [12]:
vf.shape

torch.Size([2, 600, 600])

In [13]:
def plot_vf(vf):
    fig, ax = plt.subplots(nrows=1, ncols=2)
    ax[0].imshow(vf[0].cpu().numpy())
    ax[0].set_title('X-vf')
    ax[1].imshow(vf[1].cpu().numpy())
    ax[1].set_title('Y-vf')
    plt.show()

In [14]:
plot_vf(vf)

In [15]:
mask_path = DATA_DIR / Path('train_masks/228_img_masks.png')
mask_path2 = DATA_DIR / Path('train_masks/229_img_masks.png')
image = io.imread(mask_path).astype(np.float32)
image2 = io.imread(mask_path2).astype(np.float32)

In [16]:
instance_mask_batch = torch.from_numpy(np.stack([image, image2], axis=0))[:, None]

In [17]:
instance_mask_batch.shape

torch.Size([2, 1, 250, 249])

In [18]:
aff_vf = affinity_vector_field(instance_mask_batch, kernel_size=15)

In [19]:
aff_vf.shape

torch.Size([2, 2, 250, 249])

In [21]:
plot_vf(aff_vf[0])
plot_vf(aff_vf[1])

#### Integration has 5 steps:

1. Define continuous vector field, that interpolated between the values of each pixel
2. initialize values of each pixel as the starting point
3. Integrate through to get the final values of each pixel
4. cluster the final valuse of each pixel

In [22]:
from rtseg.cellseg.numerics.interpolation.interpolate_vf import interpolate_vf
from rtseg.cellseg.numerics.integration.utils import init_values_semantic, init_values_mesh_batched
from rtseg.cellseg.numerics.integration.integrate_vf import ivp_solver

In [56]:
aff_vf.shape

torch.Size([2, 2, 250, 249])

In [57]:
continuous_vf = interpolate_vf(aff_vf[0][None, :], mode = 'bilinear')

In [58]:
pred_semantic = instance_mask_batch.clone() > 0

In [59]:
init_values = init_values_semantic(pred_semantic[0][None,:], device="cpu")

In [60]:
init_values.shape

torch.Size([2, 27635])

In [61]:
pred_semantic.shape

torch.Size([2, 1, 250, 249])

In [62]:
init_values

tensor([[ 64.,  65.,  66.,  ..., 157., 158., 159.],
        [ 10.,  10.,  10.,  ..., 249., 249., 249.]])

In [63]:
%timeit solutions = ivp_solver(continuous_vf, init_values, dx = 0.1, n_steps=5, solver="euler")

3.29 ms ± 558 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [64]:
solutions = ivp_solver(continuous_vf, init_values, dx = 0.1, n_steps=5, solver="euler")

In [65]:
final_solution = solutions[-1]

In [66]:
final_solution.shape

torch.Size([2, 27635])

In [67]:
from sklearn.cluster import DBSCAN

In [68]:
points = final_solution.T

In [69]:
%timeit clustering = DBSCAN(eps=2.25, min_samples=15, n_jobs=-1).fit(points)

59.6 ms ± 3.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [70]:
from dbscan import sklDBSCAN as DBSCAN

In [71]:
%timeit clustering = DBSCAN(eps=2.25, min_samples=15).fit(points)

15.4 ms ± 197 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [72]:
clustering = DBSCAN(eps=2.25, min_samples=15).fit(points)

In [73]:
cluster = torch.Tensor(clustering.labels_)

In [78]:
instance_segmentation = pred_semantic[0][None, :].clone().float()

In [79]:
instance_segmentation[pred_semantic[0][None,:]] = cluster + 1

In [80]:
import ncolor

In [81]:
clustered_values = ncolor.label(instance_segmentation[0][0].numpy())

In [82]:
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(clustered_values)
ax[0].set_title('clustered')
ax[1].imshow(ncolor.label(instance_mask[0][0].numpy()))
ax[1].set_title('original')
plt.show()

NameError: name 'instance_mask' is not defined

In [40]:
1 + 1

2

In [58]:
len(solutions)

5

In [59]:
fig, ax = plt.subplots(nrows=1, ncols=5)
for i in range(5):
    points_i = solutions[i].T
    clustering_i = DBSCAN(eps=2.25, min_samples=15).fit(points_i)
    cluster = torch.Tensor(clustering_i.labels_)
    instance_segmentation = pred_semantic.clone().float()
    instance_segmentation[pred_semantic] = cluster + 1
    ax[i].imshow(ncolor.label(instance_segmentation[0][0].numpy()))
    ax[i].set_title(f"Iteration: {i}")

plt.show()

4 -color algorthm failed,trying again with 5 colors. Depth 0


In [69]:
labels_dir = Path("/home/pk/Documents/rtseg/data/cellseg/omnipose/train_masks")
save_dir = Path("/home/pk/Documents/rtseg/data/cellseg/omnipose/vf_train")


In [ ]:
from rtseg.cellseg.numerics.sdf_vf import save_sdf_vf

In [72]:
save_sdf_vf(labels_dir, save_dir, kernel_size=11, vf_delimiter="_vf")

5it [00:06,  1.27s/it]


KeyboardInterrupt: 